In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Importing the dataset
data = pd.read_csv('churn.csv')


In [3]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values



In [4]:
categorical_features = data.select_dtypes(include=['object','bool']).columns.tolist()
continuous_features  = data.select_dtypes(include=['number']).columns.tolist()

#Convert categorical features to numerical
def encoding(df,categorical_features):
    
    df_cat = df[categorical_features]
    #By defaault, convert all the categories to numerical forms using Label encoder
    le = {}
    for i in range(len(df_cat.columns.tolist())):
        le[i] = LabelEncoder()
        df_cat.iloc[:,i] = le[i].fit_transform(df_cat.iloc[:,i].astype(str))

    return df_cat

encoded_data  = encoding(data,categorical_features)
data          = pd.concat([data[continuous_features],encoded_data],axis=1)
X=data.iloc[:,:-1]
y=data.iloc[:,-1]
print('Encoded data looks like this: \n',X.head())

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Encoded data looks like this: 
    account length  area code  number vmail messages  total day minutes  \
0             128        415                     25              265.1   
1             107        415                     26              161.6   
2             137        415                      0              243.4   
3              84        408                      0              299.4   
4              75        415                      0              166.7   

   total day calls  total day charge  total eve minutes  total eve calls  \
0              110             45.07              197.4               99   
1              123             27.47              195.5              103   
2              114             41.38              121.2              110   
3               71             50.90               61.9               88   
4              113             28.34              148.3              122   

   total eve charge  total night minutes  total night calls  \
0  

In [5]:


# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
# lots of high computation to ease calculation, we don't want one independent variable dominating
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [6]:
# Part 2 - Making the ANN

# Importing the Keras libraries and package
# Sequential module - initialize neural network
# Dense - layers of ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer with dropout
# Take average of input + output for units/output_dim param in Dense
# input_dim is necessary for the first layer as it was just initialized
classifier.add(Dense(15, input_dim = 20, kernel_initializer = 'glorot_uniform', activation = 'relu' ,kernel_regularizer=regularizers.l2(0.01)))
#classifier.add(Dropout(p = 0.1))

# Adding the second hidden layer with dropout
# doesn't need the input_dim params
# kernel_initializer updates weights
# activation function - rectifier
classifier.add(Dense(5, kernel_initializer = 'glorot_uniform', activation = 'relu' ,kernel_regularizer=regularizers.l2(0.001)))
#classifier.add(Dropout(p = 0.1))
classifier.add(Dense(3, kernel_initializer = 'glorot_uniform', activation = 'relu' ,kernel_regularizer=regularizers.l2(0.1)))
# Adding the output layer
# dependent variable with more than two categories (3), output_dim needs to change (e.g. 3), activation function - sufmax
classifier.add(Dense(1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid' ))

# Compiling the ANN - applying Stochastic Gradient Descent to whole ANN
# Several different SGD algorithms
# mathematical details based on the loss function
# binary_crossentropy, categorical_cross_entropy
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training Set
# X_train, y_train, Batch size, Epochs (whole training set)
classifier.fit(X_train, y_train, batch_size = 25, nb_epoch = 150)







C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:39: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/150
2666/2666 [==============================] - 1s 547us/step - loss: 1.0010 - acc: 0.7228
Epoch 2/150
2666/2666 [==============================] - 0s 163us/step - loss: 0.7535 - acc: 0.8518
Epoch 3/150
2666/2666 [==============================] - 1s 207us/step - loss: 0.5786 - acc: 0.8518
Epoch 4/150
2666/2666 [==============================] - 1s 321us/step - loss: 0.4815 - acc: 0.8518
Epoch 5/150
2666/2666 [==============================] - 1s 191us/step - loss: 0.4373 - acc: 0.8518
Epoch 6/150
2666/2666 [==============================] - 1s 222us/step - loss: 0.4129 - acc: 0.8518
Epoch 7/150
2666/2666 [==============================] - 0s 186us/step - loss: 0.3982 - acc: 0.8518
Epoch 8/150
  25/2666 [..............................] - ETA: 0s - loss: 0.3341 - acc: 0.8800

KeyboardInterrupt: 

In [144]:
# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
# Training set, see if the new data probability is right
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [145]:
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(y_test, y_pred)
cm

array([[570,   9],
       [ 31,  57]], dtype=int64)

In [146]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.95      0.98      0.97       579
          1       0.86      0.65      0.74        88

avg / total       0.94      0.94      0.94       667



In [6]:
# overfitting is when it's trained to much on the training set, less performant, test set
# and training set, high variance in, Dropout Regulariatio to reduce overfitting if needed

# Tuning the ANN, parameters learned during training (weights), stay fixed (hyperparamers - fixed, epochs, neurons)
# parameter tuning best value of these hyperparameters, GridSearchCV with k-Fold Cross Validation
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimzer):
    classifier = Sequential()
    classifier.add(Dense(6, input_dim = 20, kernel_initializer = 'glorot_uniform', activation = 'relu' ))
    classifier.add(Dense(6, kernel_initializer = 'glorot_uniform', activation = 'relu' ))
    classifier.add(Dense(1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid' ))
    classifier.compile(optimizer = optimzer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'nb_epoch': [100, 500],
              'optimzer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy', 
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

Epoch 1/1
2399/2399 [==============================] - 1s 363us/step - loss: 0.5371 - acc: 0.8495
Epoch 1/1
2399/2399 [==============================] - 1s 375us/step - loss: 0.6406 - acc: 0.6649
Epoch 1/1
2399/2399 [==============================] - 1s 382us/step - loss: 0.5106 - acc: 0.8554
Epoch 1/1
2399/2399 [==============================] - 1s 411us/step - loss: 0.5693 - acc: 0.8328
Epoch 1/1
2399/2399 [==============================] - 1s 530us/step - loss: 0.8947 - acc: 0.3022
Epoch 1/1
2399/2399 [==============================] - 1s 564us/step - loss: 0.4934 - acc: 0.8491
Epoch 1/1
2400/2400 [==============================] - 1s 588us/step - loss: 0.4958 - acc: 0.8537
Epoch 1/1
2400/2400 [==============================] - 1s 555us/step - loss: 0.5378 - acc: 0.8479
Epoch 1/1
2400/2400 [==============================] - 1s 540us/step - loss: 0.5750 - acc: 0.7667
Epoch 1/1
2400/2400 [==============================] - 1s 538us/step - loss: 0.6691 - acc: 0.6179
Epoch 1/1
2399/2399 

In [7]:
best_parameters

{'batch_size': 25, 'nb_epoch': 100, 'optimzer': 'rmsprop'}

In [42]:
# Part 2 - Making the ANN

# Importing the Keras libraries and package
# Sequential module - initialize neural network
# Dense - layers of ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer with dropout
# Take average of input + output for units/output_dim param in Dense
# input_dim is necessary for the first layer as it was just initialized
classifier.add(Dense(50, input_dim = 20, kernel_initializer = 'glorot_uniform', activation = 'relu' ,
                     kernel_regularizer=regularizers.l2(0.01)))
classifier.add(Dropout(p = 0.1))


classifier.add(Dense(21, kernel_initializer = 'glorot_uniform', activation = 'relu' ,kernel_regularizer=regularizers.l2(0.0001)))
#classifier.add(Dropout(p = 0.1))

classifier.add(Dense(1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid' ))


classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training Set
# X_train, y_train, Batch size, Epochs (whole training set)
classifier.fit(X_train, y_train, batch_size = 25, nb_epoch = 150)







C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.1)`
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/150
2666/2666 [==============================] - 2s 623us/step - loss: 0.7239 - acc: 0.8143
Epoch 2/150
2666/2666 [==============================] - 0s 158us/step - loss: 0.5333 - acc: 0.8597
Epoch 3/150
2666/2666 [==============================] - 0s 162us/step - loss: 0.4431 - acc: 0.8732
Epoch 4/150
2666/2666 [==============================] - 0s 156us/step - loss: 0.3822 - acc: 0.8961
Epoch 5/150
2666/2666 [==============================] - 0s 155us/step - loss: 0.3360 - acc: 0.9070
Epoch 6/150
2666/2666 [==============================] - 0s 172us/step - loss: 0.3144 - acc: 0.9107 0s - loss: 0.3135 - acc: 0.911
Epoch 7/150
2666/2666 [==============================] - 0s 152us/step - loss: 0.2935 - acc: 0.9126
Epoch 8/150
2666/2666 [==============================] - 0s 157us/step - loss: 0.2788 - acc: 0.9227
Epoch 9/150
2666/2666 [==============================] - 0s 167us/step - loss: 0.2715 - acc: 0.9197
Epoch 10/150
2666/2666 [==============================] - 0s 170us/st

In [43]:
# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
# Training set, see if the new data probability is right
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.35)

In [44]:
from sklearn.metrics import confusion_matrix,classification_report
cm = confusion_matrix(y_test, y_pred)
cm

array([[557,  22],
       [ 24,  64]], dtype=int64)

In [45]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.96      0.96      0.96       579
          1       0.74      0.73      0.74        88

avg / total       0.93      0.93      0.93       667

